In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd
from mlflow.models import infer_signature, ModelSignature
from mlflow.types import Schema, ColSpec

In [ ]:
# Loading data
data = datasets.load_breast_cancer()
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target,
                                                    stratify=data.target)
# Instantiating and fitting the model
model = LogisticRegression(max_iter=1000)            
model.fit(X=X_train, y=y_train)


In [ ]:
# Converting train features into a DataFrame
X_train_df = pd.DataFrame(data=X_train, columns=data.feature_names)

# Inferring the input signature
signature = infer_signature(model_input=X_train_df, 
                           model_output=model.predict(X_test))

In [ ]:
signature

In [ ]:
# Creating an input schema for the breast cancer dataset
input_schema = Schema(inputs=[ColSpec(type="double", name=feature_name) 
                              for feature_name in data.feature_names])

# Creating an output schema for the breast cancer dataset
output_schema = Schema(inputs=[ColSpec("double")])

# Creating a signature from our schemas
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [ ]:
# Example input schema for the Iris dataset
'''
input_schema = Schema(inputs=[
    ColSpec(type="double", name="sepal length (cm)"),
    ColSpec(type="double", name="sepal width (cm)"),
    ColSpec(type="double", name="petal length (cm)"),
    ColSpec(type="double", name="petal width (cm)"),
])
# Example input schema for the Iris dataset
output_schema = Schema(inputs=[ColSpec(type="long")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)
'''

In [ ]:
# Saving the model. Note the path. This will save the model under /mnt/model
input_example = X_train_df.iloc[:1]
mlflow.sklearn.save_model(sk_model=model, 
                          path="model-3", 
                          signature=signature,
                          input_example=input_example)

In [ ]:
client = mlflow.MlflowClient()


In [ ]:
# Saving the model as an artifact in a run
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
import emoji
with mlflow.start_run() as run:
    # Obtaining the ID of this run
    run_id = run.info.run_id
    # Logging our model
    model_info = mlflow.sklearn.log_model(sk_model=model, 
                             artifact_path="model",  
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=[],
                             registered_model_name='workshop')


In [ ]:
# Saving the model as an artifact in a run
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
import emoji
with mlflow.start_run() as run:
    # Obtaining the ID of this run
    run_id = run.info.run_id
    # Logging our model
    model_info = mlflow.sklearn.log_model(sk_model=model, 
                             artifact_path="model",  
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=[],
                             registered_model_name='workshop')


## How do I access custom artifacts inside the Model API Pod

1. In your workspace, add the custom file to the model artfacts. In this workbook that path is `/mnt/model`
2. Assume you have a file `initmodel.yaml` which is needed to inside the model code
3. Add this file to `/mnt/model/initmodel.yaml`
4. Register the model after the file is added
```
    model_info = mlflow.sklearn.log_model(sk_model=model, 
                             artifact_path="model",  
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=[],
                             registered_model_name='workshop')
```
5. In the the predict function of the model you want to read this file `initmodel.yaml`
6. Access as follows:
```
import os
path_to_model_artifacts = os.environ['MLFLOW_MODEL_URI']
my_file = f"{path_to_model_artifacts}/initmodel.yaml"
#And now in the prediction function access your file

```